In [1]:
import torch.nn as nn
import pytorch_lightning as pl
import torchvision.models as models

In [2]:
class Classifier(nn.Module):

    def __init__(self):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
class Resnet(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = Classifier()

    def forward(self, x):
        return self.resnet(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        data, target = train_batch
        logits = self.resnet(data)
        loss = F.binary_cross_entropy_with_logits(logits, target)
        self.log('train_loss', loss)

    def training_step(self, valid_batch, batch_idx):
        data, target = valid_batch
        logits = self.resnet(data)
        loss = F.binary_cross_entropy_with_logits(logits, target)
        self.log('valid_loss', loss)

In [4]:
model = Resnet()